In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.cpm_basic import CellularPottsModel
from scipy.spatial import ConvexHull
import matplotlib.cm as cm
import pickle
from skimage.measure import regionprops, label, perimeter as measure_perimeter
from scipy.ndimage import label as label_scipy
import pandas as pd